In [1]:
using DataFrames, DelimitedFiles, DataStructures
using GLPKMathProgInterface
using JuMP, StatsBase, CSV, LinearAlgebra, Distributions, Random
using Gurobi, BenchmarkTools

In [3]:
# using Pkg
# Pkg.add(["DataFrames", "DelimitedFiles", "DataStructures", "GLPKMathProgInterface",
#         "StatsBase", "CSV", "LinearAlgebra", "Distributions", "Random", "GLPK",
#         "BenchmarkTools", "Gurobi"])

In [2]:
function get_source()
    return CSV.read(string("Data/Projections/SaberSim/2022/Week_1.csv"), copycols=true, DataFrame)
end

get_source (generic function with 1 method)

## Function to clean inputs


In [3]:
function clean_input(year, week, version, source)
    offense = get_source()

    
    if source ==  "SaberSim"
        rename!(offense, Symbol(string("Name")) => :Player)
        rename!(offense, Symbol(string("Pos")) => :"DK Position")
        rename!(offense, Symbol(string("Opp")) => :Opponent)
        rename!(offense, Symbol(string("dk_85_percentile")) => :Projection)
        rename!(offense, Symbol(string("DFS ID")) => :DKSlateID)
        offense = offense[:, [
                :Player,
                Symbol("DK Position"),
                :Team,
                :Opponent,
                :Salary,
                :Projection,
                :DKSlateID
            ]]
        offense."DK Position" .= replace.(offense."DK Position", r",FLEX" => "")
    elseif source == "ETR"
        rename!(offense, Symbol(string("DK Salary")) => :Salary)
        rename!(offense, Symbol(string("DK Ceiling")) => :Projection)
        rename!(offense, Symbol(string("DK Ownership")) => :Ownership)
    elseif source == "Stokastic"
        rename!(offense, Symbol(string("Name")) => :Player)
        rename!(offense, Symbol(string("Position")) => :"DK Position")
        rename!(offense, Symbol(string("Fpts")) => :Projection)
    end




    offense = offense[completecases(offense), :]

 
    #cut off players that have projected points less than 5
    offense = offense[(offense."Projection".>=9) .| (offense."DK Position".== "DST"),:]
    #remove injured players

    #scores_o = copy(offense)
    #global scores_d = copy(defense)

    global num_offense = size(offense)[1]

    
    QB_indicies = Vector{Int}()
    RB_indicies = Vector{Int}()
    WR_indicies = Vector{Int}()
    TE_indicies = Vector{Int}()
    FLEX_indicies = Vector{Int}()
    DST_indicies = Vector{Int}()


   
    for i =1:num_offense
        if offense."DK Position"[i] == "QB"
            append!(QB_indicies, i)
        elseif offense."DK Position"[i] == "RB"
            append!(RB_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "WR"
            append!(WR_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "TE"
            append!(TE_indicies, i)
            append!(FLEX_indicies, i)
        elseif offense."DK Position"[i] == "DST"
            append!(DST_indicies, i)
        else
            append!(FLEX_indicies, i)
        end
    end
    
    return offense, QB_indicies, RB_indicies, WR_indicies, TE_indicies, FLEX_indicies, DST_indicies
        
        
    
end

clean_input (generic function with 1 method)

In [4]:
function calc_cor(player_matrix, temp_teams, player_cor, i, j)
    if ismissing(temp_teams[i,2])
        player_matrix[i,j] .= 0
    elseif ismissing(temp_teams[j,2])
        player_matrix[i,j] = 0
    elseif temp_teams[i,2]==temp_teams[j,2]
        try
            orig_a = player_cor[:,i]
            orig_b = player_cor[:,j]

            a = orig_a[(ismissing.(orig_b).==false) .& (ismissing.(orig_a).==false),1]
            b = orig_b[(ismissing.(orig_a).==false) .& (ismissing.(orig_b).==false),1]
            
            if (i ==1) & (j == 47)
                println(a, b)
            end
            player_matrix[i,j] = cor(a, b)
        catch err
            player_matrix[i,j] = 0
        end
    else
        player_matrix[i,j] = 0
    end
    return player_matrix[i,j]
end

calc_cor (generic function with 1 method)

In [5]:
function calc_var(player_matrix, temp_teams, player_cor, i)

    orig_a = player_cor[:,i]
    #a = orig_a[(ismissing.(orig_a).==false),1]
    a = skipmissing(orig_a)

    
    println(a)
    try
        variance = var(a)
    catch err
        variance = 0    
    end
    
    return variance
end

calc_var (generic function with 1 method)

In [6]:
function get_variance(from, to, v, max_score, year, source)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    offense2 = get_stripped_offense(from)
    offense2 = offense2[:,[:Player_Name,:Actual_FP]]
    temp = offense2
    
    for i=from+1:to-1
        offense2 = get_stripped_offense(i)
        offense2 = offense2[:,[:Player_Name,:Actual_FP]]

   
        temp = outerjoin(temp, offense2, on = :Player_Name, makeunique=true)
        
    end
    #print(temp)
    
    #temp = Tables.matrix(temp[:,2:end])
    #print(temp)
    #print(temp)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    # current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(to, v);
    current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input(year, to, v, source);
    current_offense."Player" .= replace.(current_offense."Player", r"-" => "")
    current_offense."Player" .= replace.(current_offense."Player", r"\." => "")
    current_offense."Player" .= replace.(current_offense."Player", r"'" => "")
    transform!(current_offense, :Player => ByRow(lowercase) => :Player)
    rename!(current_offense, Symbol(string("Player")) => :Player_Name)
   
   #print(current_offense) 
    
    var_array = []
    
    for i=1:length(current_offense[:,1])
        points = []
        #println(current_offense[i,1])
        #println(Matrix(filter(row -> row.Player_Name == current_offense[i,1], temp)))
        
        #println(temp)
        
        if current_offense[i,1] in temp[:,1]
            #println(current_offense[i,1])
            points_df = filter(row -> row.Player_Name == current_offense[i,1], temp)
            #println(points_df)
            for j = 2:(to-from)+1
                if ismissing(points_df[1,j]) == false
                    if Float64.(points_df[1,j]) != 0
                        append!(points, points_df[1,j])
                    end
                end
            end
            
            #println(current_offense[i,1], points)
        else            
            points = 0
        end
        
        if length(points) > 0
            variance = var(points)
        else
            variance = 0
        end
        
        if variance > 0
            append!(var_array, variance)
        else
            append!(var_array, 0)
        end
            
        #println(var_array)
    end
    
    #println(var_array)

    b = Float64.(var_array)
    
    scale = maximum(b) / max_score
    
    b = b ./ scale
    
    #print(b)
    
    return b
end


get_variance (generic function with 1 method)

In [7]:
# var_boost = get_variance(6,12,4, 33);
pwd()

"/Users/tnonet/Documents/GitHub/OptBet/dfs-nfl"

In [8]:
function get_stripped_offense(week)
    offense = CSV.read(string("Data/Actuals/2022-2023/Week ", week, "/Offense.csv"), copycols=true, DataFrame)
    rename!(offense, Symbol(string("Proj FP")) => :Projection)
    #offense = offense[offense[:,:Projection].>=3.01,:]
    
    offense."Player Name" .= replace.(offense."Player Name", r"-" => "")
    offense."Player Name" .= replace.(offense."Player Name", r"\." => "")
    offense."Player Name" .= replace.(offense."Player Name", r"'" => "")
    
    rename!(offense, Symbol(string("Player Name")) => :Player_Name)
    rename!(offense, Symbol(string("Actual FP")) => :Actual_FP)
    transform!(offense, :Player_Name => ByRow(lowercase) => :Player_Name)

    return offense
end

get_stripped_offense (generic function with 1 method)

In [9]:
function get_correlation_matrix(from, to, v, year, source)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")

    for i=from:to-1
        offense2 = get_stripped_offense(i)
        offense2 = offense2[:,[:Player_Name,:Actual_FP]]


        #print(i)
        if i==from
            temp = offense2
        else
            temp = outerjoin(temp, offense2, on = :Player_Name, makeunique=true)
        end
    end
    #print(temp)
    #cd("/Users/kevin/Dropbox (MIT)/Draft Kings/2021/2020SeasonData/")
    # current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(to, v);
    current_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input(year, to, v, source);
    current_offense."Player" .= replace.(current_offense."Player", r"-" => "")
    current_offense."Player" .= replace.(current_offense."Player", r"\." => "")
    current_offense."Player" .= replace.(current_offense."Player", r"'" => "")
    transform!(current_offense, :Player => ByRow(lowercase) => :Player)
    rename!(current_offense, Symbol(string("Player")) => :Player_Name)


    current_offense = current_offense[:,[:Player_Name,:Team]]
    #temp = temp[:,[:Player_Name,:Actual_FP]]

    #print(temp)
    #print(current_offense)
    #print(temp)
    #gold skeleton key
    temp = leftjoin(current_offense, temp, on = :Player_Name, makeunique=true)


    #temp = leftjoin(current_offense, temp, on = [:Player_Name])
    #print(temp2)
    #println(temp)
    temp = temp[:,setdiff(names(temp), [:Team])]
    temp_teams = current_offense

    temp = Matrix(temp)
    #print(temp)
    temp = permutedims(temp)
    #print(temp)


    player_list = temp[1,:]
    player_cor = temp[3:end,:]
    temp_teams = temp_teams[:,[:Player_Name,:Team]]
    #println(player_cor)



    n = size(temp_teams)[1]
    player_matrix = zeros(n,n)
    #temp_teams
    for i=1:n,j=1:n
        
        #if i in FLEX_i && j in FLEX_i && j in opp_team[i]
         #   player_matrix[i,j] = (cOffense[j,:Projection]/(9*10))
        if i in DST_i || j in DST_i
            player_matrix[i,j] = 0
        elseif i in QB_i && j in RB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        elseif j in QB_i && i in RB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j) 
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        elseif i in QB_i || j in QB_i
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
        else
            player_matrix[i,j] = calc_cor(player_matrix, temp_teams, player_cor, i, j)
            if player_matrix[i,j] < -0.2
                player_matrix[i,j] = -100
            else
                player_matrix[i,j] = player_matrix[i,j] * 0.25
            end
        end
        if isnan(player_matrix[i,j])
            player_matrix[i,j] = 0
        end
    end

    for i=1:n
        player_matrix[i,i] = 1
    end
    player_matrix[player_matrix.<-0.2] .= -100;
    return player_matrix
end

get_correlation_matrix (generic function with 1 method)

In [10]:
function get_bring_back(week, QB_i, offense)
    
    bring_back = Vector{Vector{Int}}()

    for i = 1:length(QB_i)
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[j, :Team] == offense[QB_i[i], :Opponent]) && (j in FLEX_i)
                append!(temp, j)

            end
        end
        push!(bring_back, temp)   
    end
    
    return bring_back
end

get_bring_back (generic function with 1 method)

In [11]:
function get_opposing_defense(week, QB_i, offense)
    
    opp_defense = Vector{Vector{Int}}()

    for i = 1:length(QB_i)
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[QB_i[i], :Team] == offense[j, :Opponent]) && (j in DST_i)
                append!(temp, j)

            end
        end
        push!(opp_defense, temp)   
    end
    
    return opp_defense
end

get_opposing_defense (generic function with 1 method)

In [12]:
function get_opposing_team(week, offense)
    
    opp_team = Vector{Vector{Int}}()

    for i = 1:size(t)[1]
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if (offense[i, :Team] == offense[j, :Opponent]) && i in FLEX_i && j in FLEX_i
                append!(temp, j)

            end
        end
        push!(opp_team, temp)   
    end
    
    return opp_team
end

get_opposing_team (generic function with 1 method)

In [13]:
function get_games(df)
    unq = unique(df[:,:Team])
    num_games = length(unq)
    game = Vector{Vector{Int}}()

    for i = 1:num_games
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            if j in FLEX_i || j in DST_i
                if (df[j, :Team] == unq[i]) || (df[j, :Opponent] == unq[i]) 
                    append!(temp, j)

                end
            end
        end
        push!(game, temp)   
    end
    
    return game, num_games
    
end

get_games (generic function with 1 method)

In [14]:
function get_teams(df)
    unq = unique(df[:,:Team])
    num_teams = length(unq)
    team = Vector{Vector{Int}}()

    for i = 1:num_teams
        temp = Vector{Int}()
        for j = 1:size(t)[1]
            
            #if j in FLEX_i || j in DST_i
                if (df[j, :Team] == unq[i]) 
                    append!(temp, j)

                end
           # end
        end
        push!(team, temp)   
    end
    
    return team, num_teams
    
end

get_teams (generic function with 1 method)

In [15]:
function get_idx(t)
    team_idx = Vector{Vector{Int}}()

    for i=1:size(t)[1]

        temp = Vector{Int}()

        for j=1:size(t)[2]
            if i<j
                if t[i,j] != 0
                    append!(temp, j)
                end
            end    
        end

        push!(team_idx, temp)   

    end




    #####################

    team_idx_cross = Vector{Vector{Int}}()

    for i=1:size(t)[1]

        temp = Vector{Int}()

        for j=1:size(t)[2]
            #if i<=j
                if t[i,j] > 0
                    append!(temp, j)
                end
            #end    
        end

        push!(team_idx_cross, temp)   
    end
    
    return team_idx, team_idx_cross
end

get_idx (generic function with 1 method)

In [16]:
# function late_swap_ids(week, new_version, late_teams)
#     df, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input_v2(week, new_version)

#     ids = readdlm(string("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week ", week, "/dk_ids3.csv"), ',', Int)
    
#     all_teams = []
#     for i = 1:length(ids[:,1])
#         team_i = []
#         for j = 1:9            
#             row = findall(x -> x == ids[i,j], df."DKSlateID")
#             if length(row) > 0
#                 if (df[row[1], :Team] in late_teams) == false 
#                     team_i = push!(team_i, row[1])
#                 end
#             end
#         end
#         all_teams = push!(all_teams, team_i)
#     end
    
#     return all_teams
# end

In [18]:
late_teams = ["GB", "MIN", "NYG", "TEN", "OAK", "LAC", "KC", "ARI"]

8-element Vector{String}:
 "GB"
 "MIN"
 "NYG"
 "TEN"
 "OAK"
 "LAC"
 "KC"
 "ARI"

## required input cleanings

In [21]:
year = "2022-2023"
source = "SaberSim"
week = 1
version = 1
max_score = 38
variance_start_week = -3
correlation_start_week = week -4

cOffense, QB_i, RB_i, WR_i, TE_i, FLEX_i, DST_i = clean_input(year, week, version, source);

#QB, RB, WR, TE, num_teams, num_offense, num_defense, offense, defense, offense_opp, offense_teams = clean_input(week);

t = get_correlation_matrix(correlation_start_week, week, version, year, source)



cNum_offense = length(cOffense[:,1])
#cNum_teams = length(cDefense[:,1])
num_QB = length(QB_i)
num_RB = length(RB_i)
num_WR = length(WR_i)
num_TE = length(TE_i)
num_DST = length(DST_i)
num_FLEX = length(FLEX_i)


var_boost = get_variance(variance_start_week, week, version, max_score, year, source);

In [22]:
# get a player that's opposing your QB
bring_back = get_bring_back(week, QB_i, cOffense);

In [23]:
opp_team = get_opposing_team(week, cOffense);

In [24]:
# get dataframe indexes of players on same team
# cross is indexes of players on opposing team
team_idx, team_idx_cross = get_idx(t);

In [25]:
# getting games and teams that are playing for the selected slate
game_stack, num_games = get_games(cOffense);
team_stack, num_teams = get_teams(cOffense);

## make optimization

In [32]:
function makeTeamX(num, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, var_boost)#num is what team it is from 1-100, X is current teams
    m1 = Model(Gurobi.Optimizer)
    JuMP.set_optimizer_attribute(m1, "Threads", 1)
    set_silent(m1)
    #GLPK.Optimizer
    M = 100
    max_overlap = 5.1
    max_same_player = 40
    @variable(m1, x[1:cNum_offense], Bin) #pick players
    #@variable(m1, def[1:cNum_teams], Bin) #pick players
    #@variable(m1, PB[1:cNum_offense], Bin) #player boost
    
    
    #@constraint(m1, [i=1:length(FLEX_i)], PB[FLEX_i[i]] <= sum(x[opp_team[i][j]] for j = 1:length(opp_team[i])))
    
    #@constraint(m1, [i=1:length(QB_i)], PB[QB_i[i]] == 0)
    #@constraint(m1, [i=1:length(DST_i)], PB[DST_i[i]] == 0)

    #@variable(m1, G[1:num_games], Bin)
    #@constraint(m1,[i=1:num_games], 1.5 * G[i] <= sum(x[game_stack[i][j]] for j=1:length(game_stack[i])))
    #@constraint(m1, sum(G[i] for i=1:num_games) >= 2)
    #need to make num_games and game_stack, try using uniques to get it
    
    @variable(m1, u>=0) #pick players


    @objective(m1, Max, u)

    @constraint(m1, sum(x[i]*rand_vec[i] for i=1:cNum_offense) +  # cOffense[i, :Projection] for i=1:cNum_offense) + 
                    (sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) * 9) +
                        (sum(x[i]*var_boost[i] for i=1:cNum_offense) *0.03) >= u)
                    #sum(0.1 * x[i] * x[opp_def[i][j]] for i=1:cNum_offense for j=1:length(opp_def[i])) +
                    #sum(0.1*PB[i]*cOffense[i, :Projection] for i=1:cNum_offense) +
                    

    @constraint(m1, sum(x[QB_i[i]] for i=1:num_QB) == 1)
    @constraint(m1, sum(x[RB_i[i]] for i=1:num_RB) >= 2)
    @constraint(m1, sum(x[WR_i[i]] for i=1:num_WR) >= 3)
    @constraint(m1, sum(x[TE_i[i]] for i=1:num_TE) >= 1)
    @constraint(m1, sum(x[DST_i[i]] for i=1:num_DST) == 1)
    @constraint(m1, sum(x[FLEX_i[i]] for i=1:num_FLEX) == 7)
    @constraint(m1, sum(x[i]*cOffense[i, :Salary] for i=1:cNum_offense) <= 50000)
    @constraint(m1, sum(x[i] for i=1:cNum_offense) == 9)
       
    
    # @constraint(m1,sum(x[i]*cOffense[i, :Ownership] for i=1:cNum_offense) <= 200)


    
    
    #@constraint(m1, sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) >= 1.5)
    
    #bring back
                    
    @constraint(m1,[i=1:length(QB_i)], x[QB_i[i]] - sum(x[bring_back[i][j]] for j = 1:length(bring_back[i])) <= 0)
                        
    #team stack          
    @constraint(m1,[i=1:num_teams], sum(x[team_stack[i][j]] for j=1:length(team_stack[i])) <= 4)


    
    #@constraint(m1, sum(def[i] for i=1:cNum_teams) == 1)


    if num != 1
        #for i=1:num
        @constraint(m1,[i=1:num-1], sum(t[X[i][k],team_idx_cross[X[i][k]][j]] * x[team_idx_cross[X[i][k]][j]] for k = 1:9 for j=1:length(team_idx_cross[X[i][k]])) <= 6.9)
        #end
                        
        #@constraint(m1,[i=1:num-1], sum(x[X[i][k]] for k = 1:8) <= max_overlap)
    end 
                                        
    optimize!(m1)
    return(JuMP.value.(x))
end

makeTeamX (generic function with 1 method)

In [37]:
function makeAllTeams(count, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, var_boost) #count is how many teams you want
    
    rand_vec = rand(cNum_offense,1)/25 .+1
    temp = cOffense[:, :Projection]
    rand_vec = rand_vec .* temp
    x1 = makeTeamX(1, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, var_boost);

    B = Vector{Int}()
    for i=1:length(x1)
        if x1[i] > 0.5
            append!(B,i)
        end
    end
    X = push!(X, B)
    
    for i = 2:count
        rand_vec = rand(cNum_offense,1)/25 .+1
        temp = cOffense[:, :Projection]
        rand_vec = rand_vec .* temp
        @time x1 = makeTeamX(i, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, var_boost);
        B = Vector{Int}()
        for i=1:length(x1)
            if x1[i] > 0.5
                append!(B,i)
            end
        end
        
        X = push!(X, B)
        #print(X)
        #D = hcat(D, d1)
        
        println(i)

    end

    return(X)
end

makeAllTeams (generic function with 1 method)

In [38]:
X = Vector{Vector{Int}}()

@time all_teams = makeAllTeams(150, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, var_boost);

Academic license - for non-commercial use only
Academic license - for non-commercial use only
  2.807679 seconds (284.95 k allocations: 16.231 MiB, 4.00% compilation time)
2
Academic license - for non-commercial use only
  1.228803 seconds (16.32 k allocations: 1.287 MiB)
3
Academic license - for non-commercial use only
  3.186340 seconds (19.90 k allocations: 1.400 MiB)
4
Academic license - for non-commercial use only
  5.146473 seconds (20.51 k allocations: 1.421 MiB)
5
Academic license - for non-commercial use only
  6.625901 seconds (21.02 k allocations: 1.439 MiB)
6
Academic license - for non-commercial use only
  6.088598 seconds (21.27 k allocations: 1.449 MiB)
7
Academic license - for non-commercial use only
  5.753606 seconds (21.57 k allocations: 1.461 MiB)
8
Academic license - for non-commercial use only
  9.492890 seconds (23.84 k allocations: 1.533 MiB)
9
Academic license - for non-commercial use only
 14.605843 seconds (27.28 k allocations: 1.642 MiB)
10
Academic license 

In [35]:
num_teams_making = 150


offense = cOffense
lineup2 = ""
#outfile = open(path_to_output, "w")
print_team = []



    for i = 1:num_teams_making
        lineup = ["" "" "" "" "" "" "" "" ""]
        skip = []


        for j =1:9
        
            if string(offense[all_teams[i][j],2]) in late_teams
                skip = push!(skip, all_teams[i][j])
            else
                    if all_teams[i][j] in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[all_teams[i][j],1])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[all_teams[i][j],1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[all_teams[i][j],1])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[all_teams[i][j],1])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[all_teams[i][j],1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[all_teams[i][j],1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],1])
                        end
                    elseif all_teams[i][j] in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[all_teams[i][j],1])
                        end
                    end
                #end

            end
      end
    #println(skip)
        for k in skip
                    if k in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[k,1])
                        end
                    elseif k in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[k,1])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[k,1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,1])
                        end
                    elseif k in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[k,1])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[k,1])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[k,1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,1])
                        end
                    elseif k in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[k,1])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,1])
                        end
                    elseif k in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[k,1])
                        end
                    end
                #end

        end

       print_team = push!(print_team, lineup)
    end
   #close(outfile)
#end

LoadError: UndefVarError: all_teams not defined

In [36]:
writedlm("teams_2022_2023_Sep6_2022.csv",  print_team, ',')

In [28]:
print_team

150-element Vector{Any}:
 ["Carson Wentz" "Christian McCaffrey" … "Travis Etienne Jr." "Titans"]
 ["Patrick Mahomes" "Christian McCaffrey" … "Dallas Goedert" "Dolphins"]
 ["Mitch Trubisky" "Christian McCaffrey" … "George Pickens" "Lions"]
 ["Lamar Jackson" "Christian McCaffrey" … "Dallas Goedert" "Bears"]
 ["Jared Goff" "Christian McCaffrey" … "T.J. Hockenson" "Lions"]
 ["Joe Burrow" "Najee Harris" … "Dallas Goedert" "Commanders"]
 ["Zach Wilson" "Christian McCaffrey" … "Garrett Wilson" "Bears"]
 ["Aaron Rodgers" "Christian McCaffrey" … "Aaron Jones" "Commanders"]
 ["Jalen Hurts" "Najee Harris" … "Kenneth Gainwell" "Lions"]
 ["Baker Mayfield" "Christian McCaffrey" … "Miles Sanders" "Commanders"]
 ["Jalen Hurts" "Christian McCaffrey" … "Antonio Gibson" "Dolphins"]
 ["Baker Mayfield" "Christian McCaffrey" … "David Njoku" "Dolphins"]
 ["Jalen Hurts" "Christian McCaffrey" … "D'Andre Swift" "Commanders"]
 ⋮
 ["Lamar Jackson" "Aaron Jones" … "Breece Hall" "Titans"]
 ["Trevor Lawrence" "Chris

In [29]:
print_team[1]

1×9 Matrix{String}:
 "Carson Wentz"  "Christian McCaffrey"  …  "Travis Etienne Jr."  "Titans"

In [30]:
for i=1:9
    println(offense[all_teams[10][4],1])
end

Michael Pittman Jr.
Michael Pittman Jr.
Michael Pittman Jr.
Michael Pittman Jr.
Michael Pittman Jr.
Michael Pittman Jr.
Michael Pittman Jr.
Michael Pittman Jr.
Michael Pittman Jr.


# correlation check

In [31]:
corr = 0
team_num = 0
team = "TEN"


for i = 1:length(offense[:,1])
    for j = 1:length(offense[:,1])
        #if all_teams[team_num][i]< all_teams[team_num][j] 
                if offense[i, :Team] == team && offense[j, :Team] == team
            
                #corr = t[all_teams[team_num][i],all_teams[team_num][j]]
                    println(string(offense[i,:Player])," + ", string(offense[j,:Player]), " = ", t[i,j])

                end
            
        #end
    end
end

Derrick Henry + Derrick Henry = 1.0
Derrick Henry + Ryan Tannehill = 0.1389313958823521
Derrick Henry + Robert Woods = 0.24865881765533143
Derrick Henry + Treylon Burks = -0.04525523114847672
Derrick Henry + Titans = 0.0
Derrick Henry + Austin Hooper = -100.0
Derrick Henry + Nick Westbrook-Ikhine = 0.0
Ryan Tannehill + Derrick Henry = 0.1389313958823521
Ryan Tannehill + Ryan Tannehill = 1.0
Ryan Tannehill + Robert Woods = 0.5063901498559916
Ryan Tannehill + Treylon Burks = 0.71066397278705
Ryan Tannehill + Titans = 0.0
Ryan Tannehill + Austin Hooper = -100.0
Ryan Tannehill + Nick Westbrook-Ikhine = 0.0
Robert Woods + Derrick Henry = 0.24865881765533143
Robert Woods + Ryan Tannehill = 0.5063901498559916
Robert Woods + Robert Woods = 1.0
Robert Woods + Treylon Burks = -100.0
Robert Woods + Titans = 0.0
Robert Woods + Austin Hooper = -100.0
Robert Woods + Nick Westbrook-Ikhine = 0.0
Treylon Burks + Derrick Henry = -0.04525523114847672
Treylon Burks + Ryan Tannehill = 0.71066397278705
Trey

In [32]:
length(offense[:,1])

195

In [33]:
corr = 0
team_num = 1
team = "NE"


for i = 1:9
    for j = 1:9
        if all_teams[team_num][i]< all_teams[team_num][j] 
     #corr = t[all_teams[team_num][i],all_teams[team_num][j]]
            println(string(offense[all_teams[team_num][i],1])," + ", string(offense[all_teams[team_num][j],1]), " = ", t[all_teams[team_num][i],all_teams[team_num][j]])


        end
    end
end

Christian McCaffrey + Austin Ekeler = 0.0
Christian McCaffrey + Carson Wentz = 0.0
Christian McCaffrey + Michael Pittman Jr. = 0.0
Christian McCaffrey + Travis Etienne Jr. = 0.0
Christian McCaffrey + Terry McLaurin = 0.0
Christian McCaffrey + Titans = 0.0
Christian McCaffrey + Logan Thomas = 0.0
Christian McCaffrey + Jahan Dotson = 0.0
Austin Ekeler + Carson Wentz = 0.0
Austin Ekeler + Michael Pittman Jr. = 0.0
Austin Ekeler + Travis Etienne Jr. = 0.0
Austin Ekeler + Terry McLaurin = 0.0
Austin Ekeler + Titans = 0.0
Austin Ekeler + Logan Thomas = 0.0
Austin Ekeler + Jahan Dotson = 0.0
Carson Wentz + Michael Pittman Jr. = 0.0
Carson Wentz + Travis Etienne Jr. = 0.0
Carson Wentz + Terry McLaurin = 0.9600308889162426
Carson Wentz + Titans = 0.0
Carson Wentz + Logan Thomas = 0.8758876290991978
Carson Wentz + Jahan Dotson = 0.9436130307721806
Michael Pittman Jr. + Travis Etienne Jr. = 0.0
Michael Pittman Jr. + Terry McLaurin = 0.0
Michael Pittman Jr. + Titans = 0.0
Michael Pittman Jr. + Log

## player frequnecies

In [34]:
players = []

for i = 1:num_teams_making
    for j =1:9
        #if print_team[i][j] in QB_i
            push!(players, print_team[i][j])
        #end
    end
end

frequency = []
for i in players
    
    
end

In [35]:
c = counter(players)

r = hcat(collect(keys(c)),collect(values(c)))

r[sortperm(r[:, 2], rev=true), :]

98×2 Matrix{Any}:
 "Christian McCaffrey"  100
 "Michael Pittman Jr."   86
 "Jalen Hurts"           79
 "A.J. Brown"            71
 "Mark Andrews"          58
 "Dallas Goedert"        57
 "Najee Harris"          52
 "Joe Mixon"             52
 "Jakobi Meyers"         51
 "DJ Moore"              49
 "Amon-Ra St. Brown"     42
 "Travis Kelce"          34
 "Aaron Jones"           32
 ⋮                      
 "Adam Trautman"          1
 "Terry McLaurin"         1
 "Treylon Burks"          1
 "Deebo Samuel"           1
 "Trey Lance"             1
 "Rondale Moore"          1
 "Mecole Hardman"         1
 "Evan Engram"            1
 "Kenneth Gainwell"       1
 "Trevor Lawrence"        1
 "Corey Davis"            1
 "Darren Waller"          1

In [36]:

cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 15/")



writedlm( "exposure_v3.csv",  r, ',')

LoadError: IOError: cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 15/"): no such file or directory (ENOENT)

## Print money/points

In [37]:
num_teams_making = 150


num = 9 #column 

#5 = salary
#6 = regular projection
#7 = DK value
#8 = ownership
#9 = DK ID
#10 = ceiling projection



offense = cOffense
lineup2 = ""
#outfile = open(path_to_output, "w")
print_team = []



    for i = 1:num_teams_making
        lineup = ["" "" "" "" "" "" "" "" ""]
        skip = []


        for j =1:9
        
            if string(offense[all_teams[i][j],2]) in late_teams
                skip = push!(skip, all_teams[i][j])
            else
                    if all_teams[i][j] in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[all_teams[i][j],num])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[all_teams[i][j],num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[all_teams[i][j],num])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[all_teams[i][j],num])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[all_teams[i][j],num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[all_teams[i][j],num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[all_teams[i][j],num])
                        end
                    elseif all_teams[i][j] in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[all_teams[i][j],num])
                        end
                    end
                #end

            end
      end
    #println(skip)
        for k in skip
                    if k in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[k,num])
                        end
                    elseif k in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[k,num])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[k,num])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[k,num])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[k,num])
                        end
                    end
                #end

        end

       print_team = push!(print_team, lineup)
    end
   #close(outfile)
#end

LoadError: BoundsError: attempt to access 195×7 DataFrame at index [1, 9]

In [38]:
cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 15/")



writedlm( "dk_ids.csv",  print_team, ',')

LoadError: IOError: cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 15/"): no such file or directory (ENOENT)

In [39]:
scores = []

for i =1:150
    println(sum(parse.(Float64, print_team[i])))
    append!(scores, sum(parse.(Float64, print_team[i])))
end

LoadError: BoundsError: attempt to access 0-element Vector{Any} at index [1]

In [40]:
function clean_print_teams(offense, week)
    DFN_offense = get_stripped_offense(week)
    
    offense."Player" .= replace.(offense."Player", r"-" => "")
    offense."Player" .= replace.(offense."Player", r"\." => "")
    offense."Player" .= replace.(offense."Player", r"'" => "")
    
    transform!(offense, :Player => ByRow(lowercase) => :Player)
    
    #offense[!, :Actual_FP] = 0



    for i = 1:length(offense[:,1])
        for j = 1:length(DFN_offense[:,1])
            if offense[i, :Player] == DFN_offense[j, :Player_Name]
               offense."DK Value"[i] = DFN_offense[j, :Actual_FP] 
            end
        end
    end
    

    return offense
end

clean_print_teams (generic function with 1 method)

In [41]:
offense = clean_print_teams(cOffense, 4)

LoadError: ArgumentError: "Data/Actuals/2022-2023/Week 4/Offense.csv" is not a valid file or doesn't exist

In [42]:
current_offense = get_stripped_offense(4);


LoadError: ArgumentError: "Data/Actuals/2022-2023/Week 4/Offense.csv" is not a valid file or doesn't exist

In [43]:
writedlm( "dk_ids_20.csv",  print_team, ',')

# late swap

In [44]:
function lateSwapTeamX(num, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games)#num is what team it is from 1-100, X is current teams
    m1 = Model(SCIP.Optimizer)
    set_silent(m1)
    #GLPK.Optimizer
    M = 100
    max_overlap = 5.1
    max_same_player = 40
    @variable(m1, x[1:cNum_offense], Bin) #pick players
    #@variable(m1, def[1:cNum_teams], Bin) #pick players
    #@variable(m1, PB[1:cNum_offense], Bin) #player boost
    
    
    #@constraint(m1, [i=1:length(FLEX_i)], PB[FLEX_i[i]] <= sum(x[opp_team[i][j]] for j = 1:length(opp_team[i])))
    
    #@constraint(m1, [i=1:length(QB_i)], PB[QB_i[i]] == 0)
    #@constraint(m1, [i=1:length(DST_i)], PB[DST_i[i]] == 0)

    #@variable(m1, G[1:num_games], Bin)
    #@constraint(m1,[i=1:num_games], 1.5 * G[i] <= sum(x[game_stack[i][j]] for j=1:length(game_stack[i])))
    #@constraint(m1, sum(G[i] for i=1:num_games) >= 2)
    #need to make num_games and game_stack, try using uniques to get it
    
    @variable(m1, u) #pick players


    @objective(m1, Max, u)

    @constraint(m1, sum(x[i]*rand_vec[i] for i=1:cNum_offense) +  # cOffense[i, :Projection] for i=1:cNum_offense) + 
                    (sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) * 9)>= u)
                    #sum(0.1 * x[i] * x[opp_def[i][j]] for i=1:cNum_offense for j=1:length(opp_def[i])) +
                    #sum(0.1*PB[i]*cOffense[i, :Projection] for i=1:cNum_offense) +
                    

    @constraint(m1, sum(x[QB_i[i]] for i=1:num_QB) == 1)
    @constraint(m1, sum(x[RB_i[i]] for i=1:num_RB) >= 2)
    @constraint(m1, sum(x[WR_i[i]] for i=1:num_WR) >= 3)
    @constraint(m1, sum(x[TE_i[i]] for i=1:num_TE) >= 1)
    @constraint(m1, sum(x[DST_i[i]] for i=1:num_DST) == 1)
    @constraint(m1, sum(x[FLEX_i[i]] for i=1:num_FLEX) == 7)
    @constraint(m1,sum(x[i]*cOffense[i, :Salary] for i=1:cNum_offense) <= 50000)
    @constraint(m1, sum(x[i] for i=1:cNum_offense) == 9)
                        
    @constraint(m1,sum(x[i]*cOffense."DK Ownership"[i] for i=1:cNum_offense) <= 50)
                        
    for i=1:cNum_offense
        #println(i)
        if (cOffense[i, :Team] in late_games) == false
            if isempty(current_teams[num]) == false
                #println(num)
                if i in current_teams[num]
                    #println(current_teams[num])


                    @constraint(m1, x[i] == 1)
                else
                    @constraint(m1, x[i] == 0)            
                end
            end
        end
    end


    
    
    #@constraint(m1, sum(t[i,team_idx[i][j]]*x[i]*x[team_idx[i][j]] for i=1:cNum_offense for j=1:length(team_idx[i])) >= 1.5)
    
    #bring back
                    
    @constraint(m1,[i=1:length(QB_i)], x[QB_i[i]] - sum(x[bring_back[i][j]] for j = 1:length(bring_back[i])) <= 0)
                        
    #team stack          
    @constraint(m1,[i=1:num_teams], sum(x[team_stack[i][j]] for j=1:length(team_stack[i])) <= 4)


    
    #@constraint(m1, sum(def[i] for i=1:cNum_teams) == 1)


    if num != 1
        #for i=1:num
        @constraint(m1,[i=1:num-1], sum(t[X[i][k],team_idx_cross[X[i][k]][j]] * x[team_idx_cross[X[i][k]][j]] for k = 1:9 for j=1:length(team_idx_cross[X[i][k]])) <= 7.9)
        #end
                        
        #@constraint(m1,[i=1:num-1], sum(x[X[i][k]] for k = 1:8) <= max_overlap)


    end 
                                        
    optimize!(m1)
    return(JuMP.value.(x))
end

lateSwapTeamX (generic function with 1 method)

In [45]:
function lateSwapAllTeams(count, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, current_teams, late_games) #count is how many teams you want
    
    rand_vec = rand(cNum_offense,1)/25 .+1
    temp = cOffense[:, :Projection]
    rand_vec = rand_vec .* temp
    x1 = lateSwapTeamX(1, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games);

    B = Vector{Int}()
    for i=1:length(x1)
        if x1[i] > 0.5
            append!(B,i)
        end
    end
    X = push!(X, B)
    
    for i = 2:count
        rand_vec = rand(cNum_offense,1)/25 .+1
        temp = cOffense[:, :Projection]
        rand_vec = rand_vec .* temp
        @time x1 = lateSwapTeamX(i, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, rand_vec, team_stack, num_teams, current_teams, late_games);
        B = Vector{Int}()
        for i=1:length(x1)
            if x1[i] > 0.5
                append!(B,i)
            end
        end
        
        X = push!(X, B)
        #print(X)
        #D = hcat(D, d1)
        
        println(i)

    end

    return(X)
end

lateSwapAllTeams (generic function with 1 method)

In [46]:
X = Vector{Vector{Int}}()

@time all_teams = lateSwapAllTeams(110, X, cOffense, cNum_offense, QB_i, RB_i, WR_i, TE_i, FLEX_i, num_QB, num_RB, num_WR, num_TE, num_FLEX, bring_back, team_idx, team_idx_cross, opp_team, game_stack, num_games, team_stack, num_teams, a, late_teams);

LoadError: UndefVarError: a not defined

In [47]:
a[1]

LoadError: UndefVarError: a not defined

In [48]:
cOffense[2,:]

,Player,DK Position,Team,Opponent,Salary,Projection,DKSlateID
,String31,String7,String3,String3,Int64,Float64?,Int64
2,Jalen Hurts,QB,PHI,DET,6800,30.82,23485911


In [49]:
a

LoadError: UndefVarError: a not defined

In [50]:
    ids = readdlm(string("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 12/dk_ids3.csv"), ',', Int)



LoadError: ArgumentError: Cannot open '/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 12/dk_ids3.csv': not a file

In [51]:
offense

,Player,DK Position,Team,Opponent,Salary,Projection,DKSlateID
,String31,String7,String3,String3,Int64,Float64?,Int64
1,Christian McCaffrey,RB,CAR,CLE,8500,33.6,23486004
2,Jalen Hurts,QB,PHI,DET,6800,30.82,23485911
3,Lamar Jackson,QB,BAL,NYJ,7300,30.303,23485908
4,Patrick Mahomes,QB,KC,ARI,7700,29.36,23485906
5,Jonathan Taylor,RB,IND,HOU,9100,29.515,23486000
6,Kyler Murray,QB,ARI,KC,7200,27.763,23485909
7,Justin Herbert,QB,LAC,LV,7600,26.7,23485907
8,Austin Ekeler,RB,LAC,LV,8200,29.0,23486006
9,Aaron Rodgers,QB,GB,MIN,7000,25.58,23485910


In [52]:
string(offense[all_teams[1][1],2])

"RB"

In [53]:
z = late_swap_print(all_teams, 12, 110, late_teams, cOffense)

LoadError: UndefVarError: late_swap_print not defined

In [54]:
cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 12/")



writedlm( "ls_names.csv",  z, ',')

LoadError: IOError: cd("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week 12/"): no such file or directory (ENOENT)

In [55]:
function late_swap_print(new_teams, week, num_teams, late_teams, cOffense)

    num_teams_making = num_teams
    num = 9 #column

    #5 = salary
    #6 = regular projection
    #7 = DK value
    #8 = ownership
    #9 = DK ID
    #10 = ceiling projection
    
    ids = readdlm(string("/Users/kevin/Desktop/Draftkings/2022/Data/ETR/Week ", week, "/dk_ids3.csv"), ',', Int)





    offense = cOffense
    lineup2 = ""
    #outfile = open(path_to_output, "w")
    print_team = []



    for i = 1:num_teams_making
        lineup = ["" "" "" "" "" "" "" "" ""]
        skip = []


        for j =1:9
            #println(i, " ", j)
            #if in ids, assign the same, else skip and come back to later
            in_old_teams = 0
            for t = 1:9
                #println(string(offense[new_teams[i][j],9]), ids[i,j])
                if string(offense[new_teams[i][j],9]) == string(ids[t,j]) && string(offense[new_teams[i][j],2]) in late_teams == false
                    #print("here")
                    lineup[t] = string(offense[new_teams[i][j],9])
                    in_old_teams = 0
                end
            end
            
            if in_old_teams == 0
                skip = push!(skip, new_teams[i][j])
            end
        end
    #println(skip)
        if length(skip) > 0
            for k in skip
            
                    if k in QB_i
                        if lineup[1]==""
                            lineup[1] = string(offense[k,num])
                        end
                    elseif k in RB_i
                        if lineup[2] == ""
                            lineup[2] = string(offense[k,num])
                        elseif lineup[3] == ""
                            lineup[3] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in WR_i
                        if lineup[4] == ""
                            lineup[4] = string(offense[k,num])
                        elseif lineup[5] ==""
                            lineup[5] = string(offense[k,num])
                        elseif lineup[6] == ""
                            lineup[6] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in TE_i
                        if lineup[7] == ""
                            lineup[7] = string(offense[k,num])
                        elseif lineup[8] == ""
                            lineup[8] = string(offense[k,num])
                        end
                    elseif k in DST_i
                        if lineup[9] == ""
                            lineup[9] = string(offense[k,num])
                        end
                    else
                        println(i, " ", j, " sad boi")
                    end
                #end

            end
        end

       print_team = push!(print_team, lineup)
    end
    
    return print_team
end

late_swap_print (generic function with 1 method)

In [56]:
length([])

0

In [57]:
z

LoadError: UndefVarError: z not defined

In [58]:
all_teams[12]

9-element Vector{Int64}:
   1
  12
  19
  32
  43
  49
  76
 109
 152

In [59]:
isempty(df[in(ids[11,7]).(df.DKSlateID), 2])

LoadError: UndefVarError: ids not defined

In [60]:
a = [1,2,3,4,5,6,7]

7-element Vector{Int64}:
 1
 2
 3
 4
 5
 6
 7

In [61]:
a = a./3

7-element Vector{Float64}:
 0.3333333333333333
 0.6666666666666666
 1.0
 1.3333333333333333
 1.6666666666666667
 2.0
 2.3333333333333335